In [ ]:
import numpy as np
import cv2
import os

In [ ]:
camera_number = 1
frame_count = 0
image_number = 10

#print(os.listdir())

abs_dir_path = "videos/"
VIDEO_NAME = f'out{camera_number}F.mp4'
ROW_POINTS = 5
COLUMNS_POINTS = 7
# Open the video file 
video_capture = cv2.VideoCapture(filename=f'{abs_dir_path}{VIDEO_NAME}')

# Print the number of frames in the video
number_of_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames in the video: ", number_of_frame)

In [ ]:
os.listdir()
#os.chdir('..')

In [ ]:
# Create a directory to save the screen captures named as the video file
output_dir = 'samples/' + f'out{camera_number}'
os.makedirs(output_dir, exist_ok=True)
print("Saving frames to ", output_dir)
# Remove all files in the directory
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))


In [ ]:
#findChessboard Flag test
import datetime as dt
import lib.util as util
import lib.Calibration as cal
import cv2

video_path = '...'
width = 0
height = 0

one = cal.Calibration(video_path, width, height)
skip = 500
img = []
print(f'starting skip of {skip}')
for i in range(skip):
    ret, img = one.video_capture.read()
    if not ret:
        break
print(f'frame after skip')

cv2.namedWindow("test image", cv2.WINDOW_NORMAL)

# Using resizeWindow() 
cv2.resizeWindow("test image", 1920, 1080)

# Displaying the image 
cv2.imshow("test image", img)

one.video_capture.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

#flags FAST_CHECK -> False FAST_CHECK + NORMALIZE -> False  None -> True FAST_CHECK + NORMALIZE + THRESH 
now = dt.datetime.now()
timestamp = util.getSeconds(now)
ret, imgpoints = one.extractCorners(img=img, output_dir='test/',
                                    find_flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
print(f"all flags time: {(t2 - timestamp)}")
print(ret)

now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img=img, output_dir='test/', find_flags=None)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"no flags time: {(t2 - timestamp)}")
print(ret)

##best one!
now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img=img, output_dir='test/',
                                    find_flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"no norm flag time: {(t2 - timestamp)}")
print(ret)

now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img=img, output_dir='test/', find_flags=cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"only fast flag time: {(t2 - timestamp)}")
print(ret)


In [ ]:
#COMPUTING INTRINSIC PARAMETERS TEST
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm

camera = 'out6F'
input_dir = f'samples/{camera}'
all_chessboard_sizes = [(5, 7), (5, 7), (5, 7), (5, 7), (6, 9), (6, 9), (5, 7), (6, 9), (6, 9), (0, 0), (6, 9), (5, 7),
                        (5, 7)]
chessboard_size = all_chessboard_sizes[int(re.search('\d', camera)[0]) - 1]

CHESS_WIDTH = chessboard_size[0]
CHESS_HEIGHT = chessboard_size[1]

img_list = util.loadImagesBatch(input_dir)
img_size = ((img_list[0]).shape[1], (img_list[0]).shape[0])  #width and height
imgpoints = []
objpoints = []
print(img_size)
objp = np.zeros((CHESS_WIDTH * CHESS_HEIGHT, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_WIDTH, 0:CHESS_HEIGHT].T.reshape(-1, 2)

for img in tqdm(img_list):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray, (CHESS_WIDTH, CHESS_HEIGHT), None,
                                            cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        objpoints.append(objp)
        imgpoints.append(points)

print(f'first imgpoints: {imgpoints[0]}')
#no subpixel cases

#tc std.
tc = (cv2.TERM_CRITERIA_EPS, 0.005)
print(f'tcstd{tc}')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, tc)
json_camera_matrix = {
    'ret': ret,
    'mtx': mtx.tolist(),
    'dist': dist.tolist(),
    'rvecs': [rvecs[0].tolist(), rvecs[1].tolist()],
    'tvecs': [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}std')
#tc high accuracy
tc = (cv2.TERM_CRITERIA_EPS, 0.00001)
print(f'tchigh{tc}')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, tc)
json_camera_matrix = {
    'ret': ret,
    'mtx': mtx.tolist(),
    'dist': dist.tolist(),
    'rvecs': [rvecs[0].tolist(), rvecs[1].tolist()],
    'tvecs': [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}high_acc')
#tc high accuracy and iteration
tc = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 0.005, 50)
print(f'tccombine{tc}')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, tc)
json_camera_matrix = {
    'ret': ret,
    'mtx': mtx.tolist(),
    'dist': dist.tolist(),
    'rvecs': [rvecs[0].tolist(), rvecs[1].tolist()],
    'tvecs': [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}iter')

#subpixel refinement



In [ ]:
#UNDISTORTION TEST
import numpy as np
import cv2
import os
import datetime as dt
import lib.util as util
import lib.Calibration as cal

camera = 'out6F'
image = 'chessboard10'

und_dir = f'undistorted/{camera}'
os.makedirs(und_dir, exist_ok=True)

# Remove all files in the directory
for file in os.listdir(und_dir):
    os.remove(os.path.join(und_dir, file))

filepath = f'samples/{camera}/{image}.jpg'
img = cv2.imread(filename=filepath)

#showing original image
util.showImage(img, 'original')

#refining camera matrix
path = '6Fcorners.json'
dict = util.LoadJSON(path)
h, w = img.shape[:2]
#standard values, taken from the json
std_mtx = np.array(dict['mtx'])
std_dist = np.array(dict['dist'])
print(f"numpy matrix: {std_mtx}")
new_mtx, roi = cv2.getOptimalNewCameraMatrix(std_mtx, std_dist, (w, h), 1, (w, h))

x, y, w, h = roi

#undistort the image, corrected with the new computed camera matrix
new_dist = np.array([])
und_img = cv2.undistort(src=img, cameraMatrix=std_mtx, dst=new_dist, distCoeffs=std_dist, newCameraMatrix=new_mtx)
util.showImage(und_img, 'undistorted - new camera matrix - no crop')
cv2.imwrite(f'{und_dir}/{camera}_new_mtx_no.jpg', und_img)
#print((np.size(np.array(und_img),0),np.size(np.array(und_img),1)))
# crop
crop_und_img = und_img[y:y + h, x:x + w]
util.showImage(crop_und_img, 'undistorted - new camera matrix - cropped')
cv2.imwrite(f'{und_dir}/{camera}_new_mtx.jpg', crop_und_img)
#no new camera matrix
und_img = cv2.undistort(src=img, cameraMatrix=std_mtx, dst=new_dist, distCoeffs=std_dist)
util.showImage(und_img, 'undistorted - standard camera matrix - no crop')
cv2.imwrite(f'{und_dir}/{camera}_std_mtx.jpg', und_img)
# crop 
# no_crop_und_img = und_img[y:y+h, x:x+w]
#util.showImage(und_img, 'undistorted - standard camera matrix - cropped')


In [ ]:
#image ordered calibration tests 
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm

camera = 'out2F'
input_dir = f'samples/{camera}'
all_chessboard_sizes = [(5, 7), (5, 7), (5, 7), (5, 7), (6, 9), (6, 9), (5, 7), (6, 9), (6, 9), (0, 0), (6, 9), (5, 7),
                        (5, 7)]
chessboard_size = all_chessboard_sizes[int(re.search('\d', camera)[0]) - 1]

CHESS_WIDTH = chessboard_size[0]
CHESS_HEIGHT = chessboard_size[1]

img_list = util.loadImagesBatch(input_dir)
img_list_sorted = util.loadImagesBatch(input_dir, True)

img_size = ((img_list[0]).shape[1], (img_list[0]).shape[0])  #width and height
imgpoints = []
objpoints = []
print(img_size)
objp = np.zeros((CHESS_WIDTH * CHESS_HEIGHT, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_WIDTH, 0:CHESS_HEIGHT].T.reshape(-1, 2)
tc = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 0.005, 50)

for img in tqdm(img_list):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray, (CHESS_WIDTH, CHESS_HEIGHT), None,
                                            cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        #print('found')
        objpoints.append(objp)
        imgpoints.append(points)

print(f'first imgpoints: {imgpoints[0][0]}')
print(len(imgpoints))
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, tc)
json_camera_matrix = {
    'ret': ret,
    'mtx': mtx.tolist(),
    'dist': dist.tolist(),
    'rvecs': [rvecs[0].tolist(), rvecs[1].tolist()],
    'tvecs': [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}std_fetching')

imgpoints = []
objpoints = []

for img in tqdm(img_list_sorted):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray, (CHESS_WIDTH, CHESS_HEIGHT), None,
                                            cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        objpoints.append(objp)
        imgpoints.append(points)

print(f'first imgpoints: {imgpoints[0][0]}')
print(len(imgpoints))
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, tc)
ordered_json_camera_matrix = {
    'ret': ret,
    'mtx': mtx.tolist(),
    'dist': dist.tolist(),
    'rvecs': [rvecs[0].tolist(), rvecs[1].tolist()],
    'tvecs': [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(ordered_json_camera_matrix, f'{camera}sorted_fetching')




In [ ]:
def progressBar(num: int):
    star = '*'
    slash = '-'
    bar = '['
    if num >= 20:
        print('[********************]')
    for it in range(num):
        bar = bar + star
    for it in range(20 - num):
        bar = bar + slash
    bar = bar + ']'
    return bar


def showProgressBar(iteration: int, condition: int) -> None:
    if (iteration >= condition):
        print(progressBar(20))
    else:
        print(progressBar(int((iteration) / (condition / 20))))


i = 0
while i < 100:
    showProgressBar(i, 100)
    i += 10


In [ ]:
#solvePnP TEST
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm


def computeReProjError(objpoints, imgpoints, mtx, dist, rvecs, tvecs):
    """
    method for computing multiple reprojeciton errors.
    It returns the average value of the reprojeciton errors, different from the openCv calibrateCamera ret value(which is RMSE)
        - objpoints: list of object points in the 3D camera space
        - imgpoints: list of image points in the 2D camera plane
        - mtx: camera matrix (3x3)
        - dist: distorsion values of the camera
        - rvecs: list of rotation vectors 
        - tvecs: list of translation vectors
    """
    mean_error = 0
    for i in range(np.shape(objpoints)[0]):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        mean_error += error / (np.shape(objpoints)[0])
    return mean_error


def computeMultiplePnP(objpoints, imgpoints, mtx, dist, flags=cv2.SOLVEPNP_ITERATIVE):
    tvecs = []
    rvecs = []
    for i in range(np.shape(objpoints)[0]):
        r, rvec, tvec = cv2.solvePnP(objpoints[i], imgpoints[i], mtx, dist, flags)
        rvecs.append(rvec)
        tvecs.append(tvec)
    return (rvecs, tvecs)


#Refine case of solvePnP
def computeMultipleRefinePnP(objpoints, imgpoints, mtx, dist, rvecs, tvecs):
    final_tvecs = []
    final_rvecs = []
    for i in range(np.shape(objpoints)[0]):
        print(rvecs[i])
        rvec, tvec = cv2.solvePnPRefineLM(objpoints[i], imgpoints[i], mtx, dist, rvecs[i], tvecs[i])
        final_rvecs.append(rvec)
        final_tvecs.append(tvec)
    return (final_rvecs, final_tvecs)


camera = 'out2F'
input_dir = f'samples/{camera}'
all_chessboard_sizes = [(5, 7), (5, 7), (5, 7), (5, 7), (6, 9), (6, 9), (5, 7), (6, 9), (6, 9), (0, 0), (6, 9), (5, 7),
                        (5, 7)]
chessboard_size = all_chessboard_sizes[int(re.search('\d', camera)[0]) - 1]

CHESS_WIDTH = chessboard_size[0]
CHESS_HEIGHT = chessboard_size[1]

img_list = util.loadImagesBatch(input_dir)

img_size = ((img_list[0]).shape[1], (img_list[0]).shape[0])  #width and height
imgpoints = []
objpoints = []
print(img_size)
objp = np.zeros((CHESS_WIDTH * CHESS_HEIGHT, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_WIDTH, 0:CHESS_HEIGHT].T.reshape(-1, 2)
tc = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 0.005, 50)

for img in tqdm(img_list):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray, (CHESS_WIDTH, CHESS_HEIGHT), None,
                                            cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        #print('found')
        objpoints.append(objp)
        imgpoints.append(points)
objpoints = np.array(objpoints)
imgpoints = np.array(imgpoints)

print(f'img: {np.shape(imgpoints)}\nobj: {np.shape(objpoints)}')

#Using std. calibrateCamera method as a base value for comparison
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, tc)
#print(f"ret:{ret}\nmtx:{mtx}\ndist:{dist}\nrvecs:{rvecs}\ntvecs:{tvecs}")
print(f"ret:{ret}")
print(f"rvecs0:{rvecs[0]}\ntvecs0:{tvecs[0]}")
print(f"rvecs:{rvecs}")

#using solvePnP (Levenberg-Marquardt optimization), same as calibrateCamera
pnp_rvecs, pnp_tvecs = computeMultiplePnP(objpoints, imgpoints, mtx, dist)
pnp_tvecs = np.array(pnp_tvecs)
pnp_rvecs = np.array(pnp_rvecs)
error = computeReProjError(objpoints, imgpoints, mtx, dist, pnp_rvecs, pnp_tvecs)
print(f"tvec1:{pnp_tvecs[0]}")
print(f"std. result: {error}")

#using solvePnP, coplanar object points("Infinitesimal Plane-Based Pose Estimation")
pnp_rvecs, pnp_tvecs = computeMultiplePnP(objpoints, imgpoints, mtx, dist, flags=cv2.SOLVEPNP_IPPE)
pnp_tvecs = np.array(pnp_tvecs)
pnp_rvecs = np.array(pnp_rvecs)
error = computeReProjError(objpoints, imgpoints, mtx, dist, pnp_rvecs, pnp_tvecs)
print(f"tvec2:{pnp_tvecs[0]}")
print(f"ippe. result: {error}")

###
#using "pose refinement" methods, as suggested from openCv
###

#solvePnPRefineLM(), similar to the std. method, doesn't change the result significantly

refined_rvecs, refined_tvecs = computeMultipleRefinePnP(objpoints, imgpoints, mtx, dist, rvecs, tvecs)
print(f"refined_tvec: {refined_tvecs[0]}\nrefined_rvec: {refined_rvecs[0]}")
refined_tvecs = np.array(refined_tvecs)
refined_rvecs = np.array(refined_rvecs)
error = computeReProjError(objpoints, imgpoints, mtx, dist, refined_rvecs, refined_tvecs)
print(f"refinement. result: {error}")
#solvePnPRefineVVS(), similar to the previous

# VVS_rvec, VVS_tvec = cv2.solvePnPRefineVVS(objpoints[0], imgpoints[0], mtx, dist, rvecs[0], tvecs[0] )
# print(f"VVS_tvec: {VVS_tvec}\nrefined_rvec: {VVS_rvec}")

json_camera_matrix = {
    'ret': ret,
    'mtx': mtx.tolist(),
    'dist': dist.tolist(),
    'rvecs': pnp_rvecs.tolist(),
    'tvecs': pnp_tvecs.tolist()
}



In [ ]:
import pickle

file = open('../CV-CameraCalibration/camera_parameters/without_crop/out6F.p', 'rb')
structure = pickle.load(file)

print(structure)

In [ ]:
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm


class Flag:
    f = True

    def __init__(self, value: bool):
        self.f = value

    def change(self):
        self.f = not self.f


video_path = '../videos/out1.mp4'
width = 0
height = 0


def mouseCallback(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        # print(f"event:{event}\nx:{x}\ny:{y}\nevent:{flags}")
        point = input('Enter point name:')
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, f'{point}', (x, y), font, 1, (0, 0, 0), 3)
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow('frame1', img)


def callbackButton(state, userdata):
    w_name, flag = userdata
    if flag.f:
        cv2.setMouseCallback(w_name, mouseCallback)
    else:
        cv2.setMouseCallback(w_name, lambda *args: None)
    flag.change()


def showImage(img, window_name: str):
    """
    Takes an image and show it in a fixed size window, press a button to close it in the end
        - img: An image to be shown
        - window_name: the name of the window where displaying the image
    """
    # show the original image
    w_name = window_name
    cv2.namedWindow(w_name, cv2.WINDOW_NORMAL)

    flag = Flag(True)

    cv2.createButton('select_pixel', callbackButton, (w_name, flag))

    # Using resizeWindow() 
    cv2.resizeWindow(w_name, 1920, 1080)
    cv2.imshow(w_name, img)
    cv2.waitKey(0)
    # while cv2.waitKey(33) != ord('a'):
    #     cv2.imshow(w_name, img)
    cv2.destroyWindow(window_name)


one = cal.Calibration(video_path, width, height)
skip = 15
img = []
print(f'starting skip of {skip}')
for i in range(skip):
    ret, img = one.video_capture.read()
    if not ret:
        break

showImage(img, 'frame1')


